In [ ]:
import sys
import subprocess

In [ ]:
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'Biopython'])

In [ ]:
from Bio import SeqIO
from Bio import Entrez

In [ ]:
your_genome = Entrez.efetch(db="nucleotide", id="AM747721.1", idtype="acc", rettype="gb", retmode="text")
genome = SeqIO.parse(your_genome, "gb")
SeqIO.write(genome, "your_genome.gbk", "gb")
genome_gbk = "your_genome.gbk"

In [ ]:
faa_filename = "your_genome_in_ff.faa"
input_handle  = open(genome_gbk, "r")
output_handle = open(faa_filename, "w")

for seq_record in SeqIO.parse(input_handle, "genbank") :
    for seq_feature in seq_record.features :
        try:
            if seq_feature.type=="CDS" :
                output_handle.write(">%s|%s\n" % (
                    seq_feature.qualifiers['gene'][0],
                    seq_feature.location,
                    ))
                pass
        except:
                continue
output_handle.close()
input_handle.close()

In [ ]:
records = SeqIO.parse("your_genome_in_ff.faa", "fasta")
descriptions = list([r.description for r in records])

In [ ]:
gene_of_interest = ['esmR']
match_in_list = [s for s in descriptions if any(xs in s for xs in gene_of_interest)]

In [ ]:
edit_list = ([s.replace('(+)', '') for s in match_in_list])
final_list = ([s.replace('(-)', '') for s in edit_list])
split_string = ([s.split("|", 1) for s in final_list])  
location = [i[1] for i in split_string] 
genes = [i[0] for i in split_string]

In [ ]:
location_list = ([s.replace(']', '') for s in location])
f_list = ([s.replace('[', '') for s in location_list])

In [ ]:
Gene = f_list[0]
x = Gene.split(":", 1)
location_start = int(x[0])
location_end = int(x[1])

In [ ]:
record = SeqIO.read("your_genome.gbk", "genbank")
sub_record = record[location_start:location_end]
SeqIO.write(sub_record, "your_gene.faa", "fasta")

In [ ]:
from Bio.Seq import Seq
from pathlib import Path
from Bio.SeqRecord import SeqRecord

In [ ]:
with open("OneDrive - Cardiff University/Desktop/MEP3Bassembly.fasta") as infile, open("output.txt", 'w') as outfile:
    for line in infile:
        if line.startswith(">"): continue
        outfile.write(line.strip())

string = Path('output.txt').read_text()

In [ ]:
record = SeqRecord(
    Seq(string),
)

SeqIO.write(record, "search_genome.faa", "fasta")

In [ ]:
from Bio.Blast.Applications import NcbiblastnCommandline
from Bio.Blast import NCBIXML

In [ ]:
output = NcbiblastnCommandline(query="your_gene.faa", subject="search_genome.faa", outfmt=5, out="blast_result.xml")()[0]

In [ ]:
result = open("blast_result.xml")
blast_result_record = NCBIXML.read(result)


In [ ]:
for alignment in blast_result_record.alignments:
    for hsp in alignment.hsps:
        print ('                    BLAST Alignment \n',)
        print ("Query sequence details", "\n", "\n", sub_record.features[1])
        print ("Alignment results \n")
        print ("Query length:", hsp.align_length)
        print ('e value:', hsp.expect)
        print('identity', (hsp.identities/ hsp.align_length)*100, '%')
        print('identities', hsp.identities, "/", hsp.align_length)
        print (hsp.query)
        print (hsp.match)
        print (hsp.sbjct)

In [ ]:
print("Done, time to celebrate!")